# Setup Jupyter notebook

In [1]:
from pathlib import Path
import sys

notebook_directory_parent = Path.cwd().resolve().parent.parent
print(notebook_directory_parent)
if str(notebook_directory_parent) not in sys.path:
    sys.path.append(str(notebook_directory_parent))

/cuBlackDream


# Setup to use Python libraries/modules

In [2]:
from CuISOX.DataWrangling.Kaggle.DigitRecognizer import ProcessDigitsData
from CuISOX.PyTorch.RecurrentNeuralNetworks.LSTM.Examples import (
    LSTMWithLinearOutput,
    train_LSTMWithLinearOutput_model_on_images,
    predict_on_images)
from CuISOX.utilities.configure_paths import DataPaths
from CuISOX.utilities.DataIO.KagglePaths import KagglePaths

In [ ]:
import torch

In [3]:
data_paths = DataPaths()
kaggle_paths = KagglePaths()
kaggle_data_file_paths = kaggle_paths.get_all_data_file_paths()
digit_paths = kaggle_data_file_paths["DigitRecognizer"]

# Load Data
See [Long-Short Term Memory with Pytorch](https://www.kaggle.com/code/kanncaa1/long-short-term-memory-with-pytorch)

In [4]:
training_data_paths = DataPaths.get_path_with_substring(digit_paths, "train")
training_data_path = data_paths.Kaggle() / training_data_paths[0]
print(training_data_path)

/cuBlackDream/Data/Kaggle/DigitRecognizer/digit-recognizer/train.csv


In [5]:
process_digits_data = ProcessDigitsData()

process_digits_data.parse_csv(training_data_path)
process_digits_data.load_data()

## Model Parameters and Sizes, Configuration

Batch size, epoch, and iteration

Suppose $B \equiv$ batch size.

Take the total number of samples $N$ and divide by $B$ so to get "number of batches". Given $N_{\text{iters}} \equiv$ total number of iterations, with each iteration doing 1 batch, we can get the total number of epochs.

In [6]:
batch_size = 100
# Originally, this value was 6000.
n_iters = 20000

In [ ]:
# Visualize one of the images in data set.
plt.imshow(X_features_numpy[42].reshape(28, 28))
plt.axis("off")
plt.title(str(y_targets_numpy[42]))
plt.show()

# Run Forward once

In [ ]:
training_data_iterator = enumerate(process_digits_data.training_loader)

In [7]:
input_dim = 28
hidden_dim = 100
layer_dim = 1
output_dim = 10
sequence_length = 28
# EY: 20230906, we'll use one of the physical dimensions (I'm guessing width) as the "sequence" variable. So we
# imagine that each row is an input and each successive row makes a sequence of rows of pixels of the image.
lstm_with_linear_output = LSTMWithLinearOutput(input_dim, hidden_dim, layer_dim, output_dim, sequence_length)

In [ ]:
i, (image_batch, batch_labels) = training_data_iterator.__next__()

In [ ]:
example_output, example_loss = lstm_with_linear_output.run_on_image_batch(image_batch, batch_labels)

In [ ]:
print(type(example_output))
print(type(example_loss))
print(example_output.size())
print(example_loss.size())
print(example_output.shape)
print(example_loss.shape)
print(example_output[0])
print(example_output[1])
print(example_loss.item())

In [ ]:
max_value, max_indices = torch.max(example_output.data, 1)
print(max_value)
print(max_indices)

# Train Model

In [8]:
number_of_epochs = process_digits_data.calculate_epoch(n_iters, batch_size)
# Originally, the value of number_of_epochs was 17 given n_iters = 6000
print(number_of_epochs)

59


In [9]:
%%time

lost_list, iteration_list, accuracy_list = train_LSTMWithLinearOutput_model_on_images(
    lstm_with_linear_output,
    process_digits_data,
    number_of_epochs)

print(lost_list, iteration_list, accuracy_list)

Iteration: 500, Loss: 2.244734764099121, Accuracy: 19.297618865966797
Iteration: 1000, Loss: 0.8516436219215393, Accuracy: 69.70237731933594
Iteration: 1500, Loss: 0.4007427990436554, Accuracy: 86.6547622680664
Iteration: 2000, Loss: 0.2210036665201187, Accuracy: 92.8452377319336
Iteration: 2500, Loss: 0.15652728080749512, Accuracy: 93.94047546386719
Iteration: 3000, Loss: 0.16058515012264252, Accuracy: 95.5
Iteration: 3500, Loss: 0.11539065092802048, Accuracy: 96.16666412353516
Iteration: 4000, Loss: 0.038382187485694885, Accuracy: 96.25
Iteration: 4500, Loss: 0.10289300233125687, Accuracy: 96.41666412353516
Iteration: 5000, Loss: 0.03403538465499878, Accuracy: 97.14286041259766
Iteration: 5500, Loss: 0.09679891914129257, Accuracy: 97.25
Iteration: 6000, Loss: 0.08977112919092178, Accuracy: 96.92857360839844
Iteration: 6500, Loss: 0.015737101435661316, Accuracy: 97.23809814453125
Iteration: 7000, Loss: 0.009149777702987194, Accuracy: 97.41666412353516
Iteration: 7500, Loss: 0.04447775

# Make predictions with Model

Get the testing data:

In [10]:
print(kaggle_data_file_paths)
testing_data_paths = DataPaths.get_path_with_substring(digit_paths, "test")
testing_data_path = data_paths.Kaggle() / testing_data_paths[0]
print(testing_data_path)

{'DigitRecognizer': [PosixPath('DigitRecognizer/digit-recognizer/sample_submission.csv'), PosixPath('DigitRecognizer/digit-recognizer/test.csv'), PosixPath('DigitRecognizer/digit-recognizer/train.csv')]}
/cuBlackDream/Data/Kaggle/DigitRecognizer/digit-recognizer/test.csv


In [11]:
X_numpy = ProcessDigitsData.parse_csv_no_split(testing_data_path)
test_loader = ProcessDigitsData.load_data_no_split(X_numpy, batch_size)

In [16]:
print(len(X_numpy))
print(X_numpy.shape)

28000
torch.Size([28000, 784])


In [15]:
digits_predictions = predict_on_images(lstm_with_linear_output, test_loader)
print(type(digits_predictions))
print(len(digits_predictions))
print(digits_predictions[0].size())

<class 'list'>
280
torch.Size([100])


In [ ]:
# Miscellaneous work or scratch work.
testing_data_iterator = enumerate(test_loader)
example_i, example_batch = testing_data_iterator.__next__()
print(example_i)
print(type(example_batch))
print(len(example_batch))
print(type(example_batch[0]))
print(len(example_batch[0]))